In [1]:
from pydantic import BaseModel, ValidationError, validator

# Adding extra functionality to dataclasses

## Enforced type-checking

One of the main added functionalities of using a pydantic BaseModel instead of a standard dataclass is enforced type checking at run time.

In [2]:
from datetime import datetime

test_msg_a = {'a': 1, 'b': datetime.now(), 'c': 3}

class TestMsgClassA(BaseModel):
    a: int
    b: int
    c: int

try:
    test_msg_parsed_a = TestMsgClassA(**test_msg_a)
except ValidationError as e:
    # Just doing this to ignore the traceback...
    print("Parsing failed: ", e)

Parsing failed:  1 validation error for TestMsgClassA
b
  value is not a valid integer (type=type_error.integer)


### An example with vanilla python where a date is allowed into an integer field

In [3]:
from dataclasses import dataclass

@dataclass
class TestMsgClassDC:
    a: int
    b: int
    c: int

print(TestMsgClassDC(**test_msg_a))

TestMsgClassDC(a=1, b=datetime.datetime(2022, 2, 23, 6, 49, 1, 424308), c=3)


## Easily coerce the correct type

Sometimes data coming in can be a bit messy, you can have things like integers that come through as strings, which can break your code, so you end up coercing types all over the place with things like int(...) or str(...). Pydantic will resolve this automatically, and give you the correct type you are looking for.

In [4]:
test_msg_b = {'a': 1, 'b': 2, 'c': 3}

class TestMsgClassB(BaseModel):
    a: int
    b: str
    c: float

test_msg_parsed_b = TestMsgClassB(**test_msg_b)

print(test_msg_parsed_b)

a=1 b='2' c=3.0


Notice how the types are b and c are converted to string and float respectively.

## Aliasing fields

If you're parsing data from different sources you could face things like DifferentNamingConventions, or weird symbols. Pydantic will allow you to alias fields, for example to snake case fit in better with the rest of your Python application.

In [5]:
test_msg_c = {
    "@TIMESTAMP": datetime.now(),
    "some-string": "abc",
    "AnotherValue": "def",
}

class TestMsgClassC(BaseModel):
    timestamp: datetime
    some_string: str
    another_value: str

    class Config:
        fields = {
            "timestamp": "@TIMESTAMP",
            "some_string": "some-string",
            "another_value": "AnotherValue",
        }

test_msg_parsed_c = TestMsgClassC(**test_msg_c)

print(test_msg_parsed_c)

timestamp=datetime.datetime(2022, 2, 23, 6, 49, 1, 739099) some_string='abc' another_value='def'


## Data validation

Pydantic makes it super easy to validate data coming in, and and to transform fields if necessary.

In [6]:
test_message_d = {
    "timestamp": 1645443918,
    "value": "xyz",
}

class TestMsgClassD(BaseModel):
    timestamp: datetime
    value: str

    @validator("timestamp", pre=True)
    def convert_epoch_ts_to_dt(cls, timestamp: int) -> datetime:
        return datetime.fromtimestamp(timestamp) 

test_msg_parsed_d = TestMsgClassD(**test_message_d)

print(test_msg_parsed_d)

timestamp=datetime.datetime(2022, 2, 21, 22, 45, 18) value='xyz'


This is only a trivial example, there are lots of other features for data validation and transformation!